In [9]:
%pip install torch
%pip install transformers
%pip install evaluate
%pip install scikit-learn
%pip install --upgrade datasets
import torch
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    T5Tokenizer, T5ForConditionalGeneration,
    TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer,
    EarlyStoppingCallback, DataCollatorWithPadding, DataCollatorForSeq2Seq
)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, classification_report
from datasets import Dataset, load_dataset

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Using device: {device}")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\iorfi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\iorfi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\iorfi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\iorfi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\iorfi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Using device: cpu


In [ ]:
import os
os.environ["HF_DATASETS_CACHE"] = "/root/.cache/huggingface/datasets"

def create_stratified_balanced_dataset(dataset, min_samples_per_label=15000, max_samples_per_label=50000, seed=42):
    """
  Creates a balanced dataset respect to the number of label per class
    """
    np.random.seed(seed)

    df = dataset.to_pandas()

    
    label_counts = df['label'].value_counts()
    print(f"Original distribution: {dict(label_counts)}")

    balanced_samples = []

    for label in [0, 1, 2]:  # entailment, neutral, contradiction
        label_data = df[df['label'] == label].copy()
        available_samples = len(label_data)

        if available_samples < min_samples_per_label:
            print(f"⚠️  Label {label}: only {available_samples} available samples (min: {min_samples_per_label})")
            selected_samples = label_data
        else:
            # Uniform sampling based on sequence length
            label_data['text_length'] = label_data['premise'].str.len() + label_data['hypothesis'].str.len()

            label_data['length_quartile'] = pd.qcut(label_data['text_length'], 4, labels=False, duplicates='drop')

            samples_per_quartile = min(max_samples_per_label // 4, available_samples // 4)
            quartile_samples = []

            for quartile in range(4):
                quartile_data = label_data[label_data['length_quartile'] == quartile]
                if len(quartile_data) > 0:
                    n_samples = min(samples_per_quartile, len(quartile_data))
                    sampled = quartile_data.sample(n=n_samples, random_state=seed+quartile)
                    quartile_samples.append(sampled)

            selected_samples = pd.concat(quartile_samples, ignore_index=True)
            selected_samples = selected_samples.drop(['text_length', 'length_quartile'], axis=1)

        balanced_samples.append(selected_samples)
        print(f"Label {label}: selezionati {len(selected_samples)} campioni")

    final_df = pd.concat(balanced_samples, ignore_index=True)
    final_df = final_df.sample(frac=1, random_state=seed).reset_index(drop=True)

    print(f"Final dataset: {len(final_df)} campioni")
    print(f"Final distribution: {dict(final_df['label'].value_counts())}")

    return Dataset.from_pandas(final_df)

#  LOADING AND BALANCING DATASETS 

print("=== CARICAMENTO E PREPARAZIONE DATASET MIGLIORATA ===")

# Loading dataset
esnli_dataset = load_dataset('esnli', trust_remote_code=True)

#Dataset balancing
esnli_train = create_stratified_balanced_dataset(
    esnli_dataset['train'],
    min_samples_per_label=5000,
    max_samples_per_label=20000,
    seed=42
)

esnli_val = create_stratified_balanced_dataset(
    esnli_dataset['validation'],
    min_samples_per_label=1000,
    max_samples_per_label=1500,
    seed=42
)

=== CARICAMENTO E PREPARAZIONE DATASET MIGLIORATA ===
Distribuzione originale: {0: 183416, 2: 183187, 1: 182764}
Label 0: selezionati 20000 campioni
Label 1: selezionati 20000 campioni
Label 2: selezionati 20000 campioni
Dataset bilanciato finale: 60000 campioni
Distribuzione finale: {0: 20000, 1: 20000, 2: 20000}
Distribuzione originale: {0: 3329, 2: 3278, 1: 3235}
Label 0: selezionati 1500 campioni
Label 1: selezionati 1500 campioni
Label 2: selezionati 1500 campioni
Dataset bilanciato finale: 4500 campioni
Distribuzione finale: {1: 1500, 2: 1500, 0: 1500}


In [11]:
# SETUP - BERT for Classification + T5 for Explanations

# BERT Model for Classification
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3,
    id2label={0: "Entailment", 1: "Neutral", 2: "Contradiction"},
    label2id={"Entailment": 0, "Neutral": 1, "Contradiction": 2}
).to(device)

# BERT Data Preparation Functions
def prepare_bert_data(example):
    """Prepare data for BERT classification"""
    # Combine premise and hypothesis for BERT input
    text = f"{example['premise']} [SEP] {example['hypothesis']}"
    return {
        "text": text,
        "label": example['label']
    }

# Prepare BERT data
bert_train = esnli_train.map(prepare_bert_data)
bert_val = esnli_val.map(prepare_bert_data)

print(f"Training size: {len(bert_train)}")
print(f"Validation size: {len(bert_val)}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Training size: 60000
Validation size: 4500


In [12]:
# BERT Preprocessing
def preprocess_bert(batch):
    """Tokenize data for BERT"""
    return bert_tokenizer(
        batch["text"],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )

In [13]:
# Apply preprocessing
bert_train_tokenized = bert_train.map(
    preprocess_bert,
    batched=True,
    remove_columns=["text"]
)

bert_val_tokenized = bert_val.map(
    preprocess_bert,
    batched=True,
    remove_columns=["text"]
)

# Set format for PyTorch
bert_train_tokenized.set_format("torch")
bert_val_tokenized.set_format("torch")

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

In [ ]:
# BERT Metrics
def compute_bert_metrics(eval_pred):
    """Metriche dettagliate per analisi completa"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Standard metrics
    accuracy = accuracy_score(labels, predictions)
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')

    # Class metrics
    f1_per_class = f1_score(labels, predictions, average=None)

    # Classification report 
    report = classification_report(labels, predictions, output_dict=True,
                                 target_names=['Entailment', 'Neutral', 'Contradiction'])
    return {
        'accuracy': accuracy,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_entailment': f1_per_class[0],
        'f1_neutral': f1_per_class[1],
        'f1_contradiction': f1_per_class[2],
        'precision_macro': report['macro avg']['precision'],
        'recall_macro': report['macro avg']['recall']
    }

In [15]:
# BERT Training Arguments
bert_training_args = TrainingArguments(
    output_dir="./bert_nli_high_accuracy",
    learning_rate=2e-5,  
    per_device_train_batch_size=64,  
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,  
    num_train_epochs=5, 
    weight_decay=0.01,
    warmup_ratio=0.1,
    max_grad_norm=1.0,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=4,
    seed=42,
    data_seed=42,
    report_to=None,
)

# BERT Trainer
bert_trainer = Trainer(
    model=bert_model,
    args=bert_training_args,
    train_dataset=bert_train_tokenized,
    eval_dataset=bert_val_tokenized,
    tokenizer=bert_tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=bert_tokenizer),
    compute_metrics=compute_bert_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

C:\Users\iorfi\AppData\Local\Temp\ipykernel_1916\4248213284.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  bert_trainer = Trainer(


TRAINING

In [ ]:
# Train BERT Model
print("Training BERT classifier...")
bert_trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Training BERT classifier...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: iorfidafabioaz (iorfidafabioaz-universit-degli-studi-di-milano) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save BERT model
bert_model.save_pretrained("BERT_NLI_Modelcpu")
bert_tokenizer.save_pretrained("BERT_NLI_TokenizerCPU")

print("BERT training completed and model saved!")

BERT training completed and model saved!


In [21]:
# T5 Model for Explanations
t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
t5_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base").to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#DATA PREPARATION FOR T5 MODEL
def prepare_t5_data_corrected(example):
    # Maps numeric label into text and vice versa
    label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
    label_text = label_map.get(example["label"], "neutral")

    #Create a suitable input for T%
    input_text = f"premise: {example['premise']} hypothesis: {example['hypothesis']}"

    #We are using the explanations inside e-SNLI dataset
    # e-SNLI has explanation_1, explanation_2, explanation_3
    # We will use the first explanation available
    target_text = None

    for explanation_key in ['explanation_1', 'explanation_2', 'explanation_3']:
        if explanation_key in example and example[explanation_key] and example[explanation_key].strip():
            target_text = example[explanation_key].strip()
            break

    #If there aren't any explanations, we will create an appropriate one for that pair of sentences
    if not target_text:
        if label_text == "entailment":
            target_text = f"The hypothesis logically follows from the premise. The premise provides sufficient information to conclude that the hypothesis is true. Therefore, the relationship is entailment."
        elif label_text == "contradiction":
            target_text = f"The hypothesis directly contradicts the information in the premise. The premise and hypothesis cannot both be true at the same time. Therefore, the relationship is contradiction."
        else:  # neutral
            target_text = f"The hypothesis is neither supported nor contradicted by the premise. The premise does not provide enough information to determine if the hypothesis is true or false. Therefore, the relationship is neutral."
    return {
        "input_text": input_text,
        "target_text": target_text
    }

#Preparating data for T5 model
print("Creating T5 train dataset...")
t5_train = esnli_train.map(prepare_t5_data_corrected)

print("Creating T5 validation dataset...")
t5_val = esnli_val.map(prepare_t5_data_corrected)

print(f" T5 train dataset size: {len(t5_train)}")
print(f" T5 val dataset size: {len(t5_val)}")

# Verifying data
print("\nVerifying datasets...")
sample = t5_train[0]
print(f"T5 train sample: {sample}")
print(f"Input text type: {type(sample['input_text'])}")
print(f"Target text type: {type(sample['target_text'])}")

Creating T5 train dataset...


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Creating T5 validation dataset...


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

 T5 train dataset size: 60000
 T5 val dataset size: 4500

Verifying datasets...
T5 train sample: {'premise': 'A man attempts to place basketball in hoop while surrounded by opposing teammates.', 'hypothesis': 'man competing in basketball', 'label': 0, 'explanation_1': 'Attempts to place a basketball in a hoop is a way of competing in basketball', 'explanation_2': '', 'explanation_3': '', 'input_text': 'premise: A man attempts to place basketball in hoop while surrounded by opposing teammates. hypothesis: man competing in basketball', 'target_text': 'Attempts to place a basketball in a hoop is a way of competing in basketball'}
Input text type: <class 'str'>
Target text type: <class 'str'>


In [23]:
#Size checking
try:
    print(f"Original data check:")
    print(f"t5_train size: {len(t5_train)}")
    print(f"t5_val size: {len(t5_val)}")
    print(f"Sample: {t5_train[0]}")
except NameError as e:
    print(f"Error: {e}")
    print("Make sure t5_train and t5_val datasets are loaded first!")
    raise

Original data check:
t5_train size: 60000
t5_val size: 4500
Sample: {'premise': 'A man attempts to place basketball in hoop while surrounded by opposing teammates.', 'hypothesis': 'man competing in basketball', 'label': 0, 'explanation_1': 'Attempts to place a basketball in a hoop is a way of competing in basketball', 'explanation_2': '', 'explanation_3': '', 'input_text': 'premise: A man attempts to place basketball in hoop while surrounded by opposing teammates. hypothesis: man competing in basketball', 'target_text': 'Attempts to place a basketball in a hoop is a way of competing in basketball'}


In [ ]:
# Preprocessing function for T5 Model
def preprocess_function(examples):
    # Tokenizing input sentences
    inputs = t5_tokenizer(
        examples["input_text"],
        max_length=256,
        truncation=True,
        padding=False, 
        return_tensors=None 
    )
    # Tokenizza target sentences
    with t5_tokenizer.as_target_tokenizer():
        targets = t5_tokenizer(
            examples["target_text"],
            max_length=512,
            truncation=True,
            padding=False,
            return_tensors=None
        )
    # Assegna le labels
    inputs["labels"] = targets["input_ids"]
    return inputs

In [19]:
del bert_train
del bert_val
del bert_train_tokenized
del bert_val_tokenized

In [ ]:
print("\n=== APPLYING PREPROCESSING ===")
try:
    t5_train_tokenized = t5_train.map(
        preprocess_function,
        batched=True,
        batch_size=64,
        remove_columns=['input_text', 'target_text'], 
    )

    t5_val_tokenized = t5_val.map(
        preprocess_function,
        batched=True,
        batch_size=64,
        remove_columns=['input_text', 'target_text'],
    )

    # We will remove all the colums that we are using no more
    unwanted_columns = ['premise', 'hypothesis', 'label', 'explanation_1', 'explanation_2', 'explanation_3']
    for col in unwanted_columns:
        if col in t5_train_tokenized.column_names:
            t5_train_tokenized = t5_train_tokenized.remove_columns([col])
            print(f"Removed {col} from train dataset")

        if col in t5_val_tokenized.column_names:
            t5_val_tokenized = t5_val_tokenized.remove_columns([col])
            print(f"Removed {col} from val dataset")

    print("✓ Preprocessing completed successfully")
    print(f"Final train columns: {t5_train_tokenized.column_names}")
    print(f"Final val columns: {t5_val_tokenized.column_names}")

except Exception as e:
    print(f"✗ Preprocessing failed: {e}")
    print("Debug info:")
    print(f"Tokenizer type: {type(t5_tokenizer)}")
    print(f"Available tokenizer methods: {[m for m in dir(t5_tokenizer) if not m.startswith('_')]}")
    raise

# 4. verifying data processing
print("\n=== VERIFYING PROCESSED DATA ===")
print(f"Train tokenized size: {len(t5_train_tokenized)}")
print(f"Val tokenized size: {len(t5_val_tokenized)}")

# Verifying data
sample = t5_train_tokenized[0]
print(f"Sample keys: {list(sample.keys())}")
for key, value in sample.items():
    print(f"{key}: {type(value)}, length: {len(value) if hasattr(value, '__len__') else 'N/A'}")

# 5. Data Collator
print("\n=== CREATING DATA COLLATOR ===")
data_collator = DataCollatorForSeq2Seq(
    tokenizer=t5_tokenizer,
    model=t5_model,
    label_pad_token_id=-100,
    padding=True,
    return_tensors="pt"
)

# Data collator check
print("\n=== TESTING DATA COLLATOR ===")
try:
    # Test con un piccolo batch
    test_batch = [t5_train_tokenized[i] for i in range(2)]
    collated_batch = data_collator(test_batch)

    print("✓ Data collator test successful")
    print("Batch contents:")
    for key, tensor in collated_batch.items():
        print(f"  {key}: shape {tensor.shape}, dtype {tensor.dtype}")

    # Verifica che non ci siano problemi di dimensioni
    assert collated_batch["input_ids"].dim() == 2, "input_ids should be 2D"
    assert collated_batch["labels"].dim() == 2, "labels should be 2D"
    assert collated_batch["input_ids"].shape[0] == collated_batch["labels"].shape[0], "Batch size mismatch"

    print("✓ All dimension checks passed")

except Exception as e:
    print(f"✗ Data collator test failed: {e}")
    # Debug aggiuntivo
    print("\nDebug info:")
    if 'test_batch' in locals():
        for i, item in enumerate(test_batch):
            print(f"Item {i}:")
            for key, value in item.items():
                print(f"  {key}: {type(value)}, len: {len(value) if hasattr(value, '__len__') else 'N/A'}")
    raise


=== APPLYING PREPROCESSING ===


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

C:\Users\iorfi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Removed premise from train dataset
Removed premise from val dataset
Removed hypothesis from train dataset
Removed hypothesis from val dataset
Removed label from train dataset
Removed label from val dataset
Removed explanation_1 from train dataset
Removed explanation_1 from val dataset
Removed explanation_2 from train dataset
Removed explanation_2 from val dataset
Removed explanation_3 from train dataset
Removed explanation_3 from val dataset
✓ Preprocessing completed successfully
Final train columns: ['input_ids', 'attention_mask', 'labels']
Final val columns: ['input_ids', 'attention_mask', 'labels']

=== VERIFYING PROCESSED DATA ===
Train tokenized size: 60000
Val tokenized size: 4500
Sample keys: ['input_ids', 'attention_mask', 'labels']
input_ids: <class 'list'>, length: 28
attention_mask: <class 'list'>, length: 28
labels: <class 'list'>, length: 23

=== CREATING DATA COLLATOR ===

=== TESTING DATA COLLATOR ===
✓ Data collator test successful
Batch contents:
  input_ids: shape tor

In [ ]:
# T5 Metrics
def compute_t5_metrics(eval_pred):
    """Compute metrics for T5 explanation generation"""
    predictions, labels = eval_pred

    # Replacing -100 in labels with pad_token_id before decoding
    labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)

    # Decode predictions and labels
    decoded_preds = t5_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Simple length-based evaluation
    avg_pred_length = np.mean([len(pred.split()) for pred in decoded_preds])
    avg_label_length = np.mean([len(label.split()) for label in decoded_labels])

    return {
        'avg_prediction_length': avg_pred_length,
        'avg_target_length': avg_label_length
    }

In [30]:
#Creating Trainer
print("\n=== CREATING TRAINER ===")

t5_training_args = Seq2SeqTrainingArguments(
output_dir="./t5_explanations_high_accuracy",
learning_rate=5e-5,
per_device_train_batch_size=32,
per_device_eval_batch_size=32,
num_train_epochs=4,
warmup_ratio=0.1,
weight_decay=0.01,
max_grad_norm=1.0,
logging_steps=100,
eval_strategy="epoch",
save_strategy="epoch",
load_best_model_at_end=True,
metric_for_best_model="eval_loss",
greater_is_better=False,
fp16=torch.cuda.is_available(),
predict_with_generate=True,
generation_max_length=256,
generation_num_beams=4,
seed=42,
data_seed=42,
report_to=None,
)
        
t5_trainer = Seq2SeqTrainer(
model=t5_model,
args=t5_training_args,
train_dataset=t5_train_tokenized,
eval_dataset=t5_val_tokenized,
tokenizer=t5_tokenizer,
data_collator=data_collator,
compute_metrics=compute_t5_metrics if 'compute_t5_metrics' in globals() else None,
callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("✓ Trainer created successfully")


=== CREATING TRAINER ===
✓ Trainer created successfully


C:\Users\iorfi\AppData\Local\Temp\ipykernel_1916\1448336197.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  t5_trainer = Seq2SeqTrainer(


In [ ]:
# Train T5 Model
print("Training T5 explanation generator...")
t5_trainer.train()

In [ ]:
# Save final T5 model
t5_model.save_pretrained("T5_Explanation_Finalcpu")
t5_tokenizer.save_pretrained("T5_Explanation_Tokenizer_Finalcpu")

('T5_Explanation_Tokenizer_Finalcpu\\tokenizer_config.json',
 'T5_Explanation_Tokenizer_Finalcpu\\special_tokens_map.json',
 'T5_Explanation_Tokenizer_Finalcpu\\spiece.model',
 'T5_Explanation_Tokenizer_Finalcpu\\added_tokens.json')

APPLICATION FOR CONTRADDICTION DETECTION IN POLITICAL DEBATE

In [68]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertForSequenceClassification
import torch
import pymongo
import numpy as np
from pymongo import MongoClient
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein
import torch.nn.functional as F
from typing import Dict, List, Tuple, Set
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from collections import defaultdict
import re
import json
class OptimizedPoliticalStatementAnalyzer:
    """
        Optimized analyzer which uses BERT and T5 models respectively for classification and explanation of political sentences NLI relations
    
    """
    def __init__(self, 
                 bert_model_path="./bert_nli_Final", 
                 bert_tokenizer_path="./bert_nli_tokenizer_Final",
                 t5_model_path="./T5_Explanation_Final", 
                 t5_tokenizer_path="./T5_Explanation_Tokenizer_Final"):
       
        # Device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"🚀 Using device: {self.device}")

        print("📥 Loading BERT model for classification...")
        self.bert_tokenizer = BertTokenizer.from_pretrained(bert_tokenizer_path)
        self.bert_model = BertForSequenceClassification.from_pretrained(bert_model_path).to(self.device)
        
        print("📥 Loading T5 Model for explanation...")
        self.t5_tokenizer = T5Tokenizer.from_pretrained(t5_tokenizer_path)
        self.t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_path).to(self.device)
        self.t5_model.eval()
        
        # Model for sentence embeddings (for semantic similarity)
        print("📥 Loading SentenceTransformer...")
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Zero-shot classification model (for topic classification)
        print("📥 Loading zero-shot classificator...")
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        
        # Database MongoDB
        self.client = None
        self.db = None
        self.collection = None
        
        # embeddings cache
        self.embedding_cache = {}
        
        # Predefined topics
        self.predefined_topics = [
            "economy", "healthcare", "education", "immigration", 
            "environment", "security", "labor", "technology"
        ]
        
        # NLI label map
        self.nli_label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
        
        print("✅ Inizialitation Completed!")
        
    def setup_database(self, mongo_uri, db_name="political_analysis", collection_name="statements"):
        """Setup connection MongoDB"""
        try:
            self.client = MongoClient(mongo_uri)
            self.db = self.client[db_name]
            self.collection = self.db[collection_name]
            
            # Crea indici per performance migliori
            self.collection.create_index("politician")
            self.collection.create_index("topic")
            self.collection.create_index([("politician", 1), ("topic", 1)])
            
            print(f"✅ Connesso al database {db_name}")
        except Exception as e:
            print(f"❌ Errore connessione database: {e}")
    
    def get_sentence_embedding(self, text: str):
        """Return the semantic sentence embedding, using a cache for efficiency."""
        if text in self.embedding_cache:
            return self.embedding_cache[text]

        embedding = self.sentence_model.encode([text])[0]
        self.embedding_cache[text] = embedding
        return embedding

    def find_topic_by_classification(self, statement, custom_topics=None, confidence_threshold=0.35):
        """
        Automatically classifies the topic of a sentence using an iterative mechanism.
        If the confidence score falls below the threshold, it removes the predicted topic and tries again.
        """
        topics = custom_topics.copy() if custom_topics else self.predefined_topics.copy()
        original_topics = topics.copy()  # Keep the original topics for logging
        excluded_topics = []  # Track excluded topics

        try:
            while len(topics) > 0:
                result = self.classifier(statement, topics)
                predicted_topic = result['labels'][0]
                confidence = result['scores'][0]
                
                # If confidence exceeds the threshold, return the result
                if confidence >= confidence_threshold:
                    return {
                        'topic': predicted_topic,
                        'confidence': confidence,
                        'all_scores': dict(zip(result['labels'], result['scores'])),
                        'excluded_topics': excluded_topics,
                        'iterations': len(excluded_topics) + 1
                    }
                
                # If confidence is below the threshold, remove this topic and try again
                excluded_topics.append(predicted_topic)
                topics.remove(predicted_topic)
                
                print(f"🔄 Confidence {confidence:.3f} below threshold {confidence_threshold}")
                print(f"   Excluded topic: '{predicted_topic}', {len(topics)} topics remain")
            
            # If no topic exceeds the threshold
            return {
                'topic': 'TOPIC_NOT_FOUND',
                'confidence': 0.0,
                'all_scores': {},
                'excluded_topics': excluded_topics,
                'iterations': len(excluded_topics),
                'message': f'No topic found with confidence >= {confidence_threshold}'
            }
            
        except Exception as e:
            print(f"❌ Topic classification error: {e}")
            return {
                'topic': 'unknown', 
                'confidence': 0.0, 
                'all_scores': {},
                'excluded_topics': [],
                'iterations': 0,
                'error': str(e)
            }

    def _classify_nli_relation_bert(self, premise: str, hypothesis: str) -> dict:
        """
        Classify the NLI (Natural Language Inference) relation between a premise and a hypothesis
        using the fine‑tuned BERT model.
        """
        try:
            bert_input = f"{premise} [SEP] {hypothesis}"

            # Tokenise with parameters optimised for NLI
            inputs = self.bert_tokenizer(
                bert_input,
                return_tensors="pt",
                max_length=256,
                truncation=True,
                padding=True,
                add_special_tokens=True,
                return_attention_mask=True,
                return_token_type_ids=True
            ).to(self.device)

            # Forward pass (no gradients needed for inference)
            with torch.no_grad():
                outputs = self.bert_model(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    token_type_ids=inputs.get('token_type_ids', None),
                    output_attentions=False,
                    output_hidden_states=False
                )

                logits = outputs.logits

                # Temperature scaling can smooth probabilities if desired
                temperature = 1.3
                probabilities = F.softmax(logits / temperature, dim=-1)

                predicted_class = torch.argmax(probabilities, dim=-1).item()
                confidence = probabilities[0][predicted_class].item()

            relation = self.nli_label_map[predicted_class]

            return {
                'relation': relation,
                'confidence': confidence,
                'probabilities': {
                    'entailment': probabilities[0][0].item(),
                    'neutral': probabilities[0][1].item(),
                    'contradiction': probabilities[0][2].item()
                },
                'logits': logits[0].cpu().numpy().tolist()
            }

        except Exception as e:
            print(f"❌ Error during BERT NLI classification: {e}")
            return {
                'relation': 'neutral',
                'confidence': 0.0,
                'probabilities': {
                    'entailment': 0.0,
                    'neutral': 1.0,
                    'contradiction': 0.0
                },
                'logits': [0.0, 0.0, 0.0]
            }


    def _generate_nli_explanation_t5(self, premise: str, hypothesis: str, predicted_label: str) -> str:
        """
        Generate a natural‑language explanation for the NLI prediction using the fine‑tuned T5 model.
        """
        input_text = (
            "explain nli as a teacher, make it convincing:: "
            f"premise: {premise} hypothesis: {hypothesis} label: {predicted_label}"
        )

        inputs = self.t5_tokenizer(
            input_text,
            return_tensors="pt",
            max_length=512,
            truncation=True,
            padding=True
        ).to(self.device)

        with torch.no_grad():
            outputs = self.t5_model.generate(
                inputs["input_ids"],
                min_length=30,
                max_new_tokens=150,
                num_beams=6,
                early_stopping=True,
                repetition_penalty=3.0,
                no_repeat_ngram_size=4,
                length_penalty=1.5,
                do_sample=True,
                top_k=50,
                top_p=0.92,
                temperature=0.3,
                forced_eos_token_id=self.t5_tokenizer.eos_token_id
            )

        explanation = self.t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
        return explanation

    
    def _detect_nli_relation_with_explanation(self, premise, hypothesis):
        try:
            print()
            print(f"🔍 Analisi NLI:")
            print(f"   Premise: {premise}")
            print(f"   Hypothesis: {hypothesis}")
            
            # 1. Classifiction with BERT
            bert_result = self._classify_nli_relation_bert(premise, hypothesis)
            
            # 2. Generating explanation with T5 Model
            predicted_label = bert_result['relation'].title()  # "Entailment", "Neutral", "Contradiction"
            t5_explanation = self._generate_nli_explanation_t5(premise, hypothesis, predicted_label)
            
            # Combining results
            result = {
                'relation': bert_result['relation'],
                'confidence': bert_result['confidence'],
                'probabilities': bert_result['probabilities'],
                'explanation': t5_explanation,
                'source': 'bert_t5_combined'
            }
            
            print(f"✅ Final Results: {bert_result['relation']} (conf: {bert_result['confidence']:.3f})")
            print("probabilities", result['probabilities'])
            return result
            
        except Exception as e:
            print(f"❌ ERROR in NLI detection: {e}")
            import traceback
            traceback.print_exc()
            return {
                'relation': 'neutral',
                'confidence': 0.0,
                'probabilities': {'entailment': 0.0, 'neutral': 1.0, 'contradiction': 0.0},
                'explanation': f"Errore nell'analisi: {str(e)}",
                'source': 'error'
            }

    def get_politician_statements(self, politician):
        """Retrieving all the declaration for a specific politician"""
        try:
            cursor = self.collection.find({"politician": politician})
            statements = []

            for doc in cursor:
                statements.append({
                    'statement': doc['statement'],
                    'topic': doc.get('topic'),
                    'embedding': doc.get('embedding')
                })
            return statements
            
        except Exception as e:
            print(f"❌ Errore recupero statements: {e}")
            return []
    
    def find_same_topic_statements_optimized(self, target_statement, politician, 
                                         topic_hint=None, threshold=0.5):
        """
        Optimized function for topic detection of the target sentence,
        comparing it with all the politician's statements to find those on the same topic.
        """
        # Retrieve all statements from the specified politician
        politician_statements = self.get_politician_statements(politician)

        if not politician_statements:
            print(f"No statements found for politician: {politician}")
            return []

        return self._find_by_database_topic(target_statement, politician_statements, topic_hint)

    def _find_by_database_topic(self, target_statement, politician_statements, topic_hint=None):
        """
        Finds statements from the same politician that match the topic of the target statement.
        If no topic hint is provided, it automatically classifies the target statement's topic.
        """
        # Determine topic to use
        if not topic_hint:
            topic_info = self.find_topic_by_classification(target_statement)
            target_topic = topic_info['topic']
        else:
            target_topic = topic_hint

        same_topic_statements = []

        for stmt_data in politician_statements:
            stmt_text = stmt_data['statement']
            stmt_topic = stmt_data.get('topic')

            if stmt_text == target_statement:
                continue  # Skip exact match

            # Check if topics match
            topic_match = False
            if stmt_topic and self._topics_match(target_topic, stmt_topic):
                topic_match = True
            elif not stmt_topic:
                # Classify the statement to assign a topic
                stmt_topic_info = self.find_topic_by_classification(stmt_text)
                if self._topics_match(target_topic, stmt_topic_info['topic']):
                    topic_match = True
                    stmt_topic = stmt_topic_info['topic']

            if topic_match:
                # Compute semantic similarity for ranking
                similarity = cosine_similarity(
                    [self.get_sentence_embedding(target_statement)],
                    [self.get_sentence_embedding(stmt_text)]
                )[0][0]

                same_topic_statements.append({
                    'statement': stmt_text,
                    'topic': stmt_topic,
                    'similarity': float(similarity)
                })

        return sorted(same_topic_statements, key=lambda x: x['similarity'], reverse=True)

    def _topics_match(self, topic1, topic2):
        """compare topics with fuzzy matching"""
        if not topic1 or not topic2:
            return False
        
        topic1_lower = topic1.lower().strip()
        topic2_lower = topic2.lower().strip()
        
        def topic_similarity(s1, s2, soglia=0.8):
            ratio = 1 - Levenshtein.distance(s1, s2) / max(len(s1), len(s2))
            return ratio >= soglia, ratio
        
        # Exact matching
        if topic1_lower == topic2_lower:
            return True
        elif topic_similarity(topic1_lower, topic2_lower)[0]:
            return True
        
        return False
    
    def get_available_topics_for_politician(self, politician):
        """Retrieving all the avaiable topics for a determined politician"""
        try:
            pipeline = [
                {"$match": {"politician": politician}},
                {"$group": {"_id": "$topic", "count": {"$sum": 1}}},
                {"$sort": {"count": -1}}
            ]
            
            topics = []
            for result in self.collection.aggregate(pipeline):
                if result['_id']:
                    topics.append({
                        'topic': result['_id'],
                        'statement_count': result['count']
                    })
            
            return topics
            
        except Exception as e:
            print(f"❌ Errore nel recupero topic: {e}")
            return []
    
    def add_statement(self, politician, statement, topic=None):
        """Adds declaration into database with embeddings"""
        try:
            # Generating embeddings with sentenceTransformer
            embedding = self.get_sentence_embedding(statement).tolist()
            
            # if there's not a topic, find it through classification
            if not topic:
                topic_info = self.find_topic_by_classification(statement)
                topic = topic_info['topic']
            
            document = {
                "politician": politician,
                "statement": statement,
                "topic": topic,
                "embedding": embedding
            }
            
            result = self.collection.insert_one(document)
            print(f"✅ Declaration added with ID: {result.inserted_id}")
            return result.inserted_id
            
        except Exception as e:
            print(f"❌ Errore nell'aggiunta della dichiarazione: {e}")
            return None
    
    def analyze_statement(self, politician, statement, topic_hint=None, confidence_threshold=0.5):
        """
        Optimized contradiction analysis using BERT + T5
        """
        try:
            print(f"\n🚀 Starting declaration analysis")
            print(f"👤 Politician: {politician}")
            print(f"💬 Declaration: '{statement}'")
            print(f"🏷️ Topic hint: {topic_hint}")
            print(f"📊 Confidence threshold: {confidence_threshold}")
            
            # Find similar statements on the same topic
            similar_statements = self.find_same_topic_statements_optimized(
                statement, politician, topic_hint, threshold=0.5
            )
            
            print(f"📊 Similar sentences found: {len(similar_statements)}")
            
            if not similar_statements:
                if topic_hint:
                    return {
                        'results': {'contradiction': [], 'entailment': []},
                        'stats': {'contradictions': 0, 'entailments': 0},
                        'topic_used': topic_hint,
                        'search_type': 'exact_topic',
                        'same_topic_found': 0,
                        'message': f"No sentence found for topic '{topic_hint}'"
                    }
                else:
                    topic_info = self.find_topic_by_classification(statement)
                    return {
                        'results': {'contradiction': [], 'entailment': []},
                        'stats': {'contradictions': 0, 'entailments': 0},
                        'topic_info': topic_info,
                        'search_type': 'auto_detection',
                        'same_topic_found': 0,
                        'message': f"No sentence found for topic '{topic_info['topic']}'"
                    }
            
            # Analyze NLI relations using BERT + T5
            contradictions = []
            entailments = []
            
            for stmt_info in similar_statements:
                # Use BERT + T5 to detect NLI relations with explanation
                nli_result = self._detect_nli_relation_with_explanation(stmt_info['statement'], statement) 
                
                # Filter by confidence
                if nli_result['confidence'] >= confidence_threshold:
                    entry = {
                        'statement': stmt_info['statement'],
                        'similarity': stmt_info['similarity'],
                        'topic': stmt_info.get('topic'),
                        'nli_relation': nli_result['relation'],
                        'nli_confidence': nli_result['confidence'],
                        'nli_probabilities': nli_result['probabilities'],
                        'explanation': nli_result['explanation'],
                        'model_source': nli_result['source']
                    }
                    
                    if nli_result['relation'] == 'contradiction':
                        contradictions.append(entry)
                    elif nli_result['relation'] == 'entailment':
                        entailments.append(entry)
                else:
                    print(f"⚠️ Confidence too low ({nli_result['confidence']:.3f} < {confidence_threshold})")
            
            print(f"\n📈 Total contradictions: {len(contradictions)}")
            print(f"📈 Total entailments: {len(entailments)}")
            
            # Prepare response
            if topic_hint:
                result = {
                    'results': {
                        'contradiction': contradictions,
                        'entailment': entailments
                    },
                    'stats': {
                        'contradictions': len(contradictions),
                        'entailments': len(entailments)
                    },
                    'topic_used': topic_hint,
                    'search_type': 'exact_topic',
                    'same_topic_found': len(similar_statements),
                    'confidence_threshold': confidence_threshold
                }
            else:
                topic_info = self.find_topic_by_classification(statement)
                result = {
                    'results': {
                        'contradiction': contradictions,
                        'entailment': entailments
                    },
                    'stats': {
                        'contradictions': len(contradictions),
                        'entailments': len(entailments)
                    },
                    'topic_info': topic_info,
                    'search_type': 'auto_detection',
                    'same_topic_found': len(similar_statements),
                    'confidence_threshold': confidence_threshold
                }
            
            print(f"🏁 Analysis completed")
            return result
            
        except Exception as e:
            print(f"❌ Error in analysis: {e}")
            import traceback
            traceback.print_exc()
            return {
                'results': {'contradiction': [], 'entailment': []},
                'stats': {'contradictions': 0, 'entailments': 0},
                'error': str(e)
            }

    def bulk_import_statements(self, csv_file_path):
        """Import declarations from CSV file with embedding generation"""
        try:
            df = pd.read_csv(csv_file_path)
            
            documents = []
            for _, row in df.iterrows():
                # emebedding generation
                embedding = self.get_sentence_embedding(row['statement']).tolist()
                
                # Topic classification if missed
                topic = row.get('topic')
                if pd.isna(topic) or not topic:
                    topic_info = self.find_topic_by_classification(row['statement'])
                    topic = topic_info['topic']
                
                document = {
                    "politician": row['politician'],
                    "statement": row['statement'],
                    "topic": topic,
                    "embedding": embedding
                }
                documents.append(document)
            
            # adding batch
            if documents:
                result = self.collection.insert_many(documents)
                print(f"✅ Importate {len(result.inserted_ids)} dichiarazioni")
            
        except Exception as e:
            print(f"❌ Error during the import: {e}")
    
    def get_politicians_statistics(self):
        """Politician statistics from the db"""
        try:
            pipeline = [
                {"$group": {
                    "_id": "$politician",
                    "total_statements": {"$sum": 1},
                    "topics": {"$addToSet": "$topic"}
                }},
                {"$project": {
                    "politician": "$_id",
                    "total_statements": 1,
                    "unique_topics": {"$size": "$topics"},
                    "_id": 0
                }}
            ]  
            return list(self.collection.aggregate(pipeline))
            
        except Exception as e:
            print(f"❌ Error while computing statistics: {e}")
            return []
    
    def close(self):
        """Close db connection"""
        if self.client:
            self.client.close()
            print("✅ Connessione database chiusa")
    
    def _create_nli_test_cases(self) -> Dict[str, List[Tuple[str, str, str]]]:
        """
        Creates premise-hypothesis pairs organized by NLI test category.
        Format: (premise, hypothesis, expected_label)
        Labels: 'entailment', 'contradiction', 'neutral'
        """
        test_cases = {
            'mutual_exclusion': [
                ("Mario Rossi says he will never increase taxes for self-employed workers",
                "Mario Rossi will increase taxes for self-employed workers",
                "contradiction"),

                ("Giulia Bianchi promises not to cut public healthcare anymore",
                "Giulia Bianchi will make cuts to public healthcare",
                "contradiction"),

                ("Antonio Verdi says he will support universal basic income",
                "Antonio Verdi opposes universal basic income",
                "contradiction"),

                ("Antonio Verdi supports universal basic income",
                "Antonio Verdi wants to give citizens a monthly payment",
                "entailment"),

                ("Maria Neri says Italy will become the European leader in solar energy by 2030",
                "Maria Neri wants to improve Italy’s energy autonomy",
                "neutral"),
            ],

            'numerical_differences': [
                ("Mario Rossi promises to reduce taxes by 15%",
                "Mario Rossi will reduce taxes by 20%",
                "contradiction"),

                ("Giulia Bianchi will increase teachers' salaries by 10%",
                "Giulia Bianchi will increase teachers' salaries by 5%",
                "contradiction"),

                ("Mario Rossi will invest 5 billion in renewable energy",
                "Mario Rossi will invest 5 billion in renewable energy",
                "entailment"),

                ("Maria Neri will introduce a 15% flat tax",
                "Maria Neri plans to apply a simplified tax system",
                "neutral"),

                ("Antonio Verdi will cut emissions by 30% in 5 years",
                "Antonio Verdi will cut emissions by 35% in 5 years",
                "contradiction"),
            ],

            'logical_entailment': [
                ("Mario Rossi says he will hire 10,000 new doctors and nurses",
                "Mario Rossi will increase healthcare personnel",
                "entailment"),

                ("Giulia Bianchi will increase teachers' salaries by 10%",
                "Giulia Bianchi will improve teachers' economic conditions",
                "entailment"),

                ("Maria Neri wants to introduce a climate education program in schools",
                "Maria Neri cares about climate change",
                "entailment"),

                ("Luca Gialli will implement digital ID systems",
                "Luca Gialli will increase funding for public transport",
                "neutral"),

                ("Antonio Verdi will install new security cameras in high-risk areas",
                "Antonio Verdi believes cameras violate privacy",
                "contradiction"),
            ],

            'neutral_relations': [
                ("Mario Rossi will reduce taxes for the middle class",
                "Mario Rossi will invest in renewable energy",
                "neutral"),

                ("Giulia Bianchi will increase teachers' salaries",
                "Giulia Bianchi will build new schools",
                "neutral"),

                ("Antonio Verdi wants to abolish IRAP tax for small businesses",
                "Antonio Verdi will install security cameras",
                "neutral"),

                ("Maria Neri will reduce waiting lists in hospitals",
                "Maria Neri will plant one million trees",
                "neutral"),

                ("Luca Gialli will eliminate the gender pay gap",
                "Luca Gialli will digitize public administration",
                "neutral"),
            ],

            'specificity_tests': [
                ("Mario Rossi will reduce taxes for the middle class by 15%",
                "Mario Rossi will reduce taxes",
                "entailment"),

                ("Giulia Bianchi will reduce CO2 emissions by 40% by 2030",
                "Giulia Bianchi will commit to the environment",
                "entailment"),

                ("Antonio Verdi will hire more law enforcement officers",
                "Antonio Verdi will hire exactly 1,200 police officers",
                "neutral"),

                ("Maria Neri will improve public healthcare",
                "Maria Neri will introduce free mental health consultations in all schools",
                "neutral"),

                ("Luca Gialli will invest in green urban projects",
                "Luca Gialli refuses to support ecological plans",
                "contradiction"),
            ],

            'paraphrase_tests': [
                ("Mario Rossi will never increase taxes for self-employed workers",
                "Mario Rossi will keep taxes stable for freelancers",
                "entailment"),

                ("Giulia Bianchi wants to hire 10,000 new teachers",
                "Giulia Bianchi will increase the teaching staff by ten thousand units",
                "entailment"),

                ("Antonio Verdi will fight petty crime in big cities",
                "Antonio Verdi will address small urban crimes",
                "entailment"),

                ("Maria Neri will plant one million trees in Italian cities",
                "Maria Neri will implement a massive urban forestation campaign",
                "entailment"),

                ("Luca Gialli will promote gender equality in companies",
                "Luca Gialli opposes measures for equal pay",
                "contradiction"),
            ],

            'complex_negation': [
                ("Mario Rossi says that the tax burden is too high and should be reduced",
                "Mario Rossi thinks that the tax burden is not excessive",
                "contradiction"),

                ("Giulia Bianchi states that youth unemployment is the absolute priority",
                "Giulia Bianchi does not consider youth unemployment a priority",
                "contradiction"),

                ("Antonio Verdi will not tolerate citizens' safety being compromised",
                "Antonio Verdi accepts that there are security problems",
                "contradiction"),

                ("Maria Neri denies wanting to privatize healthcare",
                "Maria Neri supports public healthcare",
                "entailment"),

                ("Luca Gialli does not exclude changing the pension reform in the future",
                "Luca Gialli has confirmed the current pension law is final",
                "contradiction"),
            ],

            'temporal_conditional': [
                ("Mario Rossi will reduce taxes within the next fiscal year",
                "Mario Rossi will reduce taxes immediately",
                "neutral"),

                ("Giulia Bianchi will increase teachers' salaries starting next school year",
                "Giulia Bianchi has already increased teachers' salaries",
                "neutral"),

                ("Antonio Verdi will propose new agreements with countries of origin",
                "Antonio Verdi has already signed agreements with countries of origin",
                "neutral"),

                ("Maria Neri will build 5 new hospitals by 2026",
                "Maria Neri plans to expand the hospital network in the coming years",
                "entailment"),

                ("Luca Gialli will only approve tax reforms if GDP grows",
                "Luca Gialli will approve tax reforms regardless of economic conditions",
                "contradiction"),
            ]
        }
        return test_cases
        
    def run_nli_evaluation(self, custom_test_cases=None, confidence_threshold=0.5, 
                            save_results=True, results_file="nli_evaluation_results.json"):
            """
            Runs a comprehensive evaluation of the NLI system on predefined test cases
            
            Args:
                custom_test_cases: Dictionary with custom tests (optional)
                confidence_threshold: Minimum confidence threshold to consider a prediction
                save_results: Whether to save results to file
                results_file: File name to save results
                
            Returns:
                Dict with detailed metrics and analysis by category
            """
            print("🚀 Starting comprehensive NLI evaluation...")
            
            # Use predefined or custom tests
            if custom_test_cases:
                test_cases = custom_test_cases
            else:
                test_cases = self._create_nli_test_cases()
            
            # Structures to collect results
            all_predictions = []
            all_true_labels = []
            category_results = {}
            detailed_results = []
            
            # Label mapping
            label_to_id = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
            id_to_label = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
            
            print(f"📊 Tests configured for {len(test_cases)} categories")
            
            # Run tests for each category
            for category, cases in test_cases.items():
                print(f"\n🔍 Testing category: {category} ({len(cases)} cases)")
                
                category_predictions = []
                category_true_labels = []
                category_details = []
                
                for i, (premise, hypothesis, expected_label) in enumerate(cases):
                    print(f"  Test {i+1}/{len(cases)}: {expected_label}")
                    
                    try:
                        # NLI classification
                        result = self._detect_nli_relation_with_explanation(premise, hypothesis)
                        
                        predicted_label = result['relation']
                        confidence = result['confidence']
                        
                        # Record results
                        all_predictions.append(label_to_id[predicted_label])
                        all_true_labels.append(label_to_id[expected_label])
                        category_predictions.append(label_to_id[predicted_label])
                        category_true_labels.append(label_to_id[expected_label])
                        
                        # Details for analysis
                        test_detail = {
                            'category': category,
                            'premise': premise,
                            'hypothesis': hypothesis,
                            'expected_label': expected_label,
                            'predicted_label': predicted_label,
                            'confidence': confidence,
                            'probabilities': result['probabilities'],
                            'explanation': result['explanation'],
                            'correct': predicted_label == expected_label,
                            'high_confidence': confidence >= confidence_threshold
                        }
                        
                        detailed_results.append(test_detail)
                        category_details.append(test_detail)
                        
                        # Immediate feedback
                        status = "✅" if predicted_label == expected_label else "❌"
                        print(f"    {status} Pred: {predicted_label} (conf: {confidence:.3f}) | Expected: {expected_label}")
                        
                    except Exception as e:
                        print(f"    ❌ ERROR in test {i+1}: {e}")
                        # Add neutral prediction in case of error
                        all_predictions.append(1)  # neutral
                        all_true_labels.append(label_to_id[expected_label])
                        category_predictions.append(1)
                        category_true_labels.append(label_to_id[expected_label])
                
                # Calculate metrics per category
                if category_predictions:
                    category_accuracy = accuracy_score(category_true_labels, category_predictions)
                    category_precision, category_recall, category_f1, _ = precision_recall_fscore_support(
                        category_true_labels, category_predictions, average='weighted', zero_division=0
                    )
                    
                    category_results[category] = {
                        'accuracy': float(category_accuracy),
                        'precision': float(category_precision),
                        'recall': float(category_recall),
                        'f1_score': float(category_f1),
                        'total_cases': len(cases),
                        'correct_predictions': sum(1 for d in category_details if d['correct']),
                        'high_confidence_predictions': sum(1 for d in category_details if d['high_confidence']),
                        'details': category_details
                    }
                    
                    print(f"  📈 Category {category}: Accuracy={category_accuracy:.3f}, F1={category_f1:.3f}")
            
            # Calculate global metrics
            if all_predictions and all_true_labels:
                overall_accuracy = accuracy_score(all_true_labels, all_predictions)
                overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(
                    all_true_labels, all_predictions, average='weighted', zero_division=0
                )
                
                # Confusion matrix
                conf_matrix = confusion_matrix(all_true_labels, all_predictions)
                
                # Detailed classification report
                class_report = classification_report(
                    all_true_labels, all_predictions, 
                    target_names=['entailment', 'neutral', 'contradiction'],
                    output_dict=True, zero_division=0
                )
                
                # Analysis by label
                label_analysis = {}
                for label_id, label_name in id_to_label.items():
                    true_positives = sum(1 for t, p in zip(all_true_labels, all_predictions) if t == label_id and p == label_id)
                    false_positives = sum(1 for t, p in zip(all_true_labels, all_predictions) if t != label_id and p == label_id)
                    false_negatives = sum(1 for t, p in zip(all_true_labels, all_predictions) if t == label_id and p != label_id)
                    
                    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
                    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
                    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
                    
                    label_analysis[label_name] = {
                        'precision': precision,
                        'recall': recall,
                        'f1_score': f1,
                        'true_positives': true_positives,
                        'false_positives': false_positives,
                        'false_negatives': false_negatives,
                        'support': sum(1 for t in all_true_labels if t == label_id)
                    }
            
            # Compile final results
            evaluation_results = {
                'overall_metrics': {
                    'accuracy': float(overall_accuracy),
                    'precision': float(overall_precision),
                    'recall': float(overall_recall),
                    'f1_score': float(overall_f1),
                    'total_test_cases': len(all_predictions),
                    'confidence_threshold': confidence_threshold
                },
                'confusion_matrix': conf_matrix.tolist(),
                'classification_report': class_report,
                'label_analysis': label_analysis,
                'category_results': category_results,
                'detailed_results': detailed_results,
                'error_analysis': self._analyze_errors(detailed_results),
            }
            
            # Save results if requested
            if save_results:
                try:
                    with open(results_file, 'w', encoding='utf-8') as f:
                        json.dump(evaluation_results, f, indent=2, ensure_ascii=False)
                    print(f"💾 Results saved to: {results_file}")
                except Exception as e:
                    print(f"❌ Save error: {e}")
            
            # Print final summary
            self._print_evaluation_summary(evaluation_results)
            
            return evaluation_results
        
    def _analyze_errors(self, detailed_results):
            """Analyzes errors to identify common patterns"""
            errors = [r for r in detailed_results if not r['correct']]
            
            if not errors:
                return {'total_errors': 0, 'error_patterns': {}}
            
            error_patterns = {
                'by_category': defaultdict(int),
                'by_expected_label': defaultdict(int),
                'by_predicted_label': defaultdict(int),
                'low_confidence_errors': 0,
                'high_confidence_errors': 0,
                'confusion_pairs': defaultdict(int)
            }
            
            for error in errors:
                error_patterns['by_category'][error['category']] += 1
                error_patterns['by_expected_label'][error['expected_label']] += 1
                error_patterns['by_predicted_label'][error['predicted_label']] += 1
                
                if error['confidence'] < 0.5:
                    error_patterns['low_confidence_errors'] += 1
                else:
                    error_patterns['high_confidence_errors'] += 1
                
                confusion_pair = f"{error['expected_label']} -> {error['predicted_label']}"
                error_patterns['confusion_pairs'][confusion_pair] += 1
            
            return {
                'total_errors': len(errors),
                'error_rate': len(errors) / len(detailed_results),
                'error_patterns': dict(error_patterns)
            }
        
    def _print_evaluation_summary(self, results):
            """Prints a summary of evaluation results"""
            print("\n" + "="*80)
            print("📊 NLI EVALUATION SUMMARY")
            print("="*80)
            
            overall = results['overall_metrics']
            print(f"🎯 GLOBAL METRICS:")
            print(f"   • Accuracy:  {overall['accuracy']:.3f}")
            print(f"   • Precision: {overall['precision']:.3f}")
            print(f"   • Recall:    {overall['recall']:.3f}")
            print(f"   • F1-Score:  {overall['f1_score']:.3f}")
            print(f"   • Total tests: {overall['total_test_cases']}")
            
            print(f"\n📈 PERFORMANCE BY CATEGORY:")
            for category, metrics in results['category_results'].items():
                print(f"\n🔸 {category}")
                print(f"   • Accuracy:  {metrics['accuracy']:.3f}")
                print(f"   • Precision: {metrics['precision']:.3f}")
                print(f"   • Recall:    {metrics['recall']:.3f}")
                print(f"   • F1-Score:  {metrics['f1_score']:.3f}")
                print(f"   • Total cases: {metrics['total_cases']}")
                print(f"   • Correct predictions: {metrics['correct_predictions']}")
                print(f"   • High-confidence predictions: {metrics['high_confidence_predictions']}")
                    
            print(f"\n🏷️  PERFORMANCE BY LABEL:")
            for label, metrics in results['label_analysis'].items():
                print(f"   • {label:15s}: P={metrics['precision']:.3f}, R={metrics['recall']:.3f}, F1={metrics['f1_score']:.3f} (support: {metrics['support']})")
            
            error_analysis = results['error_analysis']
            print(f"\n❌ ERROR ANALYSIS:")
            print(f"   • Total errors: {error_analysis['total_errors']} ({error_analysis['error_rate']:.1%})")
            print(f"   • High confidence errors: {error_analysis['error_patterns']['high_confidence_errors']}")
            print(f"   • Low confidence errors: {error_analysis['error_patterns']['low_confidence_errors']}")
    
            print("="*80)

In [ ]:
# GLOBAL VARIABLES (modify them as needed)
mongo_uri = "mongodb://localhost:27017/"
db_name = "political_analysis"  # <-- Put here the name of your database
collection_name = "statements"  # <-- Put here the name of your collection
bert_model_path = "./BERT_NLI_Model"  # Path to your BERT model
bert_tokenizer_path = "./BERT_NLI_Tokenizer"  # Path to the BERT tokenizer
t5_model_path = "./T5_Explanation_Final"  # Path to your T5 model
t5_tokenizer_path = "./T5_Explanation_Tokenizer_Final"  # Path to the T5 tokenizer


In [ ]:
# Only for the first time (import into the db the sentence for testing the application)
if __name__ == "__main__":
    # Inizializza l'analizzatore con i tuoi modelli addestrati
    analyzer = OptimizedPoliticalStatementAnalyzer(
        bert_model_path=bert_model_path,  
        bert_tokenizer_path=bert_tokenizer_path,  
        t5_model_path=t5_model_path,  
        t5_tokenizer_path=t5_tokenizer_path 
    )
         # Configura database
    analyzer.setup_database(
        mongo_uri=mongo_uri, 
        db_name=db_name, #<-- put here the neame of your db
        collection_name=collection_name #<-- put here the neame of your collection
    )

analyzer.bulk_import_statements("C:/Users/iorfi/Desktop/NLP/political_statements_sample.csv") #<-- put here the path of the db csv file

🚀 Using device: cpu
📥 Loading BERT model for classification...
📥 Loading T5 Model for explanation...
📥 Loading SentenceTransformer...
📥 Loading zero-shot classificator...


Device set to use cpu


✅ Inizializzazione completata!
✅ Connesso al database political_analysis


In [ ]:
if __name__ == "__main__":
    # Initialize the analyzer with your trained models
    analyzer = OptimizedPoliticalStatementAnalyzer(
        bert_model_path=bert_model_path,
        bert_tokenizer_path=bert_tokenizer_path,  
        t5_model_path=t5_model_path, 
        t5_tokenizer_path=t5_tokenizer_path 
    )
    # Setup database
    analyzer.setup_database(
        mongo_uri=mongo_uri, 
        db_name=db_name,
        collection_name=collection_name
    )

    print("=== OPTIMIZED POLITICAL STATEMENT ANALYZER ===")
    print("Uses semantic embeddings and EXACT search for specified topics\n")

    choice = input("Choose option:\n1. Check statement consistency\n2. Add statement to database\n3. Show available topics for politician\nChoice: ")

    if choice == '1':
        politician = input("Politician name: ")

        # Show available topics
        available_topics = analyzer.get_available_topics_for_politician(politician)
        if available_topics:
            print(f"\n📋 Available topics for {politician}:")
            for topic_info in available_topics[:10]:  # Show first 10
                print(f"  • {topic_info['topic']} ({topic_info['statement_count']} statements)")

        statement = input("\nStatement: ")
        topic_hint = input("Statement topic (leave empty for automatic detection): ").strip() or None

        if topic_hint:
            print(f"🎯 Searching database for topic: '{topic_hint}'")
        else:
            print("🔍 Automatically detecting topic and searching contradictions...")

        # Optimized analysis
        results = analyzer.analyze_statement(politician, statement, topic_hint)

        print(f"\n📊 ANALYSIS RESULTS")
        print(f"Statement: '{statement}'")

        if results.get('search_type') == 'exact_topic':
            print(f"Search type: EXACT for topic '{results['topic_used']}'")
        else:
            if 'topic_info' in results:
                topic_info = results['topic_info']
                print(f"Automatically detected topic: {topic_info['topic']} (confidence: {topic_info['confidence']:.2f})")

        print(f"Similar statements found on same topic: {results.get('same_topic_found', 0)}")
        print(f"Contradictions found: {results['stats']['contradictions']}")

        if 'message' in results:
            print(f"ℹ️ {results['message']}")

        if results['results']['contradiction']:
            print(f"\n⚠️ CONTRADICTIONS FOUND:")
            for i, contra in enumerate(results['results']['contradiction'], 1):
                print(f"\n{i}. Previous statement:")
                print(f"   '{contra['statement']}'")
                print(f"   New statement:")
                print(f"   '{statement}'")
                print(f"   Topic: {contra.get('topic', 'N/A')}")
                print(f"   Semantic similarity: {contra['similarity']:.2f}")
                print(f"   Logical relation: {contra.get('nli_relation', 'N/A')}")
                print(f"   Explanation: {contra['explanation']}")
        else:
            print(f"\n✅ No contradictions found")

        save_choice = input(f"\nDo you want to save this statement in the database? (y/N): ")
        if save_choice.lower() == 'y':
            if topic_hint:
                topic_to_save = topic_hint
            else:
                topic_to_save = results.get('topic_info', {}).get('topic')
            analyzer.add_statement(politician, statement, topic_to_save)
            print("Statement saved!")

    elif choice == '2':
        politician = input("Politician name: ")
        statement = input("Statement: ")
        topic = input("Topic (leave empty for automatic classification): ").strip() or None

        results = analyzer.analyze_statement(politician, statement, topic)
        statement_id = analyzer.add_statement(politician, statement, topic)
        if topic:
            topic_to_save = topic
        else:
            topic_to_save = results.get('topic_info', {}).get('topic')
        if statement_id:
            print("✅ Statement added successfully!")
        else:
            print("❌ Error adding the statement")

    elif choice == '3':
        politician = input("Politician name: ")
        topics = analyzer.get_available_topics_for_politician(politician)
        if topics:
            print(f"\n📋 Available topics for {politician}:")
            for topic_info in topics:
                print(f"  • {topic_info['topic']} ({topic_info['statement_count']} statements)")
        else:
            print(f"❌ No topics found for {politician}")

    else:
        print("Invalid choice")

    print(f"\n👋 Analysis completed!")


🚀 Using device: cpu
📥 Loading BERT model for classification...
📥 Loading T5 Model for explanation...
📥 Loading SentenceTransformer...
📥 Loading zero-shot classificator...


Device set to use cpu


✅ Inizialitation Completed!
✅ Connesso al database political_analysis
=== OPTIMIZED POLITICAL STATEMENT ANALYZER ===
Uses semantic embeddings and EXACT search for specified topics


📋 Available topics for Mario Rossi:
  • healthcare (3 statements)
  • economy (3 statements)
  • environment (3 statements)
  • immigration (1 statements)
  • security (1 statements)
🔍 Automatically detecting topic and searching contradictions...

🚀 Starting declaration analysis
👤 Politician: Mario Rossi
💬 Declaration: 'We will disinvest from renewable energy'
🏷️ Topic hint: None
📊 Confidence threshold: 0.5
📊 Similar sentences found: 3

🔍 Analisi NLI:
   Premise: We will invest 5 billion euros in renewable energy over the next three years.
   Hypothesis: We will disinvest from renewable energy
✅ Final Results: contradiction (conf: 0.504)
probabilities {'entailment': 0.33273643255233765, 'neutral': 0.163133442401886, 'contradiction': 0.5041301250457764}

🔍 Analisi NLI:
   Premise: We will propose a five-year

In [59]:
if __name__ == "__main__":
    # Inizializza l'analizzatore con i tuoi modelli addestrati
    analyzer = OptimizedPoliticalStatementAnalyzer(
        bert_model_path="./BERT_NLI_Model",  # Percorso del tuo modello BERT
        bert_tokenizer_path="./BERT_NLI_Tokenizer",  # Percorso del tokenizer BERT
        t5_model_path="./T5_Explanation_Model",  # Percorso del tuo modello T5
        t5_tokenizer_path="./T5_Explanation_Tokenizer"  # Percorso del tokenizer T5
    )
         # Configura database
    analyzer.setup_database(
        mongo_uri="mongodb://localhost:27017/", 
        db_name="political_analysis"
    )
results = analyzer.run_nli_evaluation()

🚀 Using device: cpu
📥 Loading BERT model for classification...
📥 Loading T5 Model for explanation...
📥 Loading SentenceTransformer...
📥 Loading zero-shot classificator...


Device set to use cpu


✅ Inizialitation Completed!
✅ Connesso al database political_analysis
🚀 Starting comprehensive NLI evaluation...
📊 Tests configured for 8 categories

🔍 Testing category: mutual_exclusion (5 cases)
  Test 1/5: contradiction
🔍 Analisi NLI:
   Premise: Mario Rossi says he will never increase taxes for self-employed workers
   Hypothesis: Mario Rossi will increase taxes for self-employed workers
✅ Final Results: contradiction (conf: 0.453)
probabilities {'entailment': 0.23972633481025696, 'neutral': 0.30754706263542175, 'contradiction': 0.4527266323566437}
    ✅ Pred: contradiction (conf: 0.453) | Expected: contradiction
  Test 2/5: contradiction
🔍 Analisi NLI:
   Premise: Giulia Bianchi promises not to cut public healthcare anymore
   Hypothesis: Giulia Bianchi will make cuts to public healthcare
✅ Final Results: entailment (conf: 0.594)
probabilities {'entailment': 0.5943519473075867, 'neutral': 0.2288503497838974, 'contradiction': 0.17679770290851593}
    ❌ Pred: entailment (conf: 0.594